In [2]:
!pip install xmltodict

In [3]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

import pandas as pd
import xmltodict
import json

##### 공공데이터활용지원센터_보건복지부 코로나19 감염 현황
> 활용신청하기
https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376

In [4]:
key='6llbCBdG4KZubbR6o3fEOB%2FTp%2BdYOfIA9AwBVAmiHLwiA8QXc3bEX39hmpS6s2rWk%2F7GDBAzKZFDz%2BuIBVjmdQ%3D%3D'
url = f'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={key}&'
queryParams = urlencode({ quote_plus('pageNo') : 1, 
                          quote_plus('numOfRows') : 10,
                          quote_plus('startCreateDt') : '20210119',
                          quote_plus('endCreateDt') : '20210120'})
url2 = url + queryParams
response = urlopen(url2)
# print(type(response)) # HTTPSresponse 
results = response.read().decode("utf-8")
# print(type(results))   # str
results_to_json = xmltodict.parse(results)
data = json.loads(json.dumps(results_to_json))
print(type(data))   # dic
print(data)   

<class 'dict'>
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'accDefRate': '1.4573418044', 'accExamCnt': '5192119', 'accExamCompCnt': '5043978', 'careCnt': '12028', 'clearCnt': '60180', 'createDt': '2021-01-20 09:38:16.549', 'deathCnt': '1300', 'decideCnt': '73508', 'examCnt': '148141', 'resutlNegCnt': '4970470', 'seq': '392', 'stateDt': '20210120', 'stateTime': '00:00', 'updateDt': '2021-04-20 15:23:25.562'}, {'accDefRate': '1.4661941407', 'accExamCnt': '5140315', 'accExamCompCnt': '4985970', 'careCnt': '12353', 'clearCnt': '59468', 'createDt': '2021-01-19 09:39:49.588', 'deathCnt': '1283', 'decideCnt': '73104', 'examCnt': '154345', 'resutlNegCnt': '4912866', 'seq': '391', 'stateDt': '20210119', 'stateTime': '00:00', 'updateDt': '2021-04-20 15:23:34.538'}]}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '2'}}}


In [5]:
data

{'response': {'body': {'items': {'item': [{'accDefRate': '1.4573418044',
      'accExamCnt': '5192119',
      'accExamCompCnt': '5043978',
      'careCnt': '12028',
      'clearCnt': '60180',
      'createDt': '2021-01-20 09:38:16.549',
      'deathCnt': '1300',
      'decideCnt': '73508',
      'examCnt': '148141',
      'resutlNegCnt': '4970470',
      'seq': '392',
      'stateDt': '20210120',
      'stateTime': '00:00',
      'updateDt': '2021-04-20 15:23:25.562'},
     {'accDefRate': '1.4661941407',
      'accExamCnt': '5140315',
      'accExamCompCnt': '4985970',
      'careCnt': '12353',
      'clearCnt': '59468',
      'createDt': '2021-01-19 09:39:49.588',
      'deathCnt': '1283',
      'decideCnt': '73104',
      'examCnt': '154345',
      'resutlNegCnt': '4912866',
      'seq': '391',
      'stateDt': '20210119',
      'stateTime': '00:00',
      'updateDt': '2021-04-20 15:23:34.538'}]},
   'numOfRows': '10',
   'pageNo': '1',
   'totalCount': '2'},
  'header': {'resultCode

In [19]:
 df = data['response']['body']['items']['item']
 df2 = pd.DataFrame(df)
 df2

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.4573418044,5192119,5043978,12028,60180,2021-01-20 09:38:16.549,1300,73508,148141,4970470,392,20210120,00:00,2021-04-20 15:23:25.562
1,1.4661941407,5140315,4985970,12353,59468,2021-01-19 09:39:49.588,1283,73104,154345,4912866,391,20210119,00:00,2021-04-20 15:23:34.538


In [ ]:
 #df3 = df2.loc[['createDt'], ['decideCnt'],['clearCnt'], ['careCnt'], ['deathCnt']]
 df3 = df2.loc[:, ['stateDt', 'decideCnt', 'clearCnt', 'careCnt', 'deathCnt' ]]
 df3

In [41]:
df4 = df3.rename(columns={'stateDt':'날자', 'decideCnt':'누적확진자', 'clearCnt':'격리해제환자','careCnt':'치료중환자','deathCnt':'사망자수' })
df4

,날자,누적확진자,격리해제환자,치료중환자,사망자수
0,20210120,73508,60180,12028,1300
1,20210119,73104,59468,12353,1283


위의 데이터를 분석해서 아래의 형태의 CSV 파일을 만들자
```
날짜,누적확진자,격리해제환자,치료중환자,사망자수
20210119,73104,59468,12353,1283
20210120,73508,60180,12028,1300

```